### Demo of OCTO+ and other object placement methods

Install

In [ ]:
!git clone https://github.com/octo-pearl/octo-pearl.git
%cd octo-pearl
%pip install -qe .

Download Weights

In [ ]:
!mkdir -p weights
!wget -q -O weights/sam_vit_h_4b8939.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!wget -q -O weights/ram_plus_swin_large_14m.pth https://huggingface.co/xinyu1205/recognize-anything-plus-model/resolve/main/ram_plus_swin_large_14m.pth
!wget -q -O weights/groundingdino_swint_ogc.pth https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

Input

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
OBJECT_TO_PLACE = "plate"
IMAGE_PATH = "assets/test_img.jpg"

image = Image.open(IMAGE_PATH)
plt.imshow(image)
plt.axis('off')
plt.show()

Stage 1: Image Understanding (Generate and filter a list of "tags", or objects, in the image)

In [ ]:
from octo_pearl.placement.tagging import get_tags_scp, get_tags_gpt4v, get_tags_ram
# tags = get_tags_scp(image)
# tags = get_tags_gpt4v(image)
tags = get_tags_ram(image, threshold_multiplier=0.8)
tags

In [ ]:
from octo_pearl.placement.filtering import filter_tags_clipseg, filter_tags_vilt, filter_tags_gdino
# filtered_tags = filter_tags_clipseg(image, tags)
# filtered_tags = filter_tags_vilt(image, tags)
filtered_tags = filter_tags_gdino(image, tags)
filtered_tags

Stage 2: Reasoning (Select which tag the chosen object should be placed on)

In [ ]:
# TODO
selected_object = "plate"

Stage 3: Locating (Select a 2D location corresponding to the selected tag)

In [ ]:
from octo_pearl.placement.locating import get_location_clipseg, get_location_gsam
# x, y = get_location_clipseg(image, selected_object)
x, y = get_location_gsam(image, selected_object)
plt.imshow(image)
plt.scatter(x, y, c="red", s=50)
plt.axis('off')
plt.show()